<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [GTSRB5] - Full convolutions
<!-- DESC --> Episode 5 : A lot of models, a lot of datasets and a lot of results.
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
  - Try multiple solutions
  - Design a generic and batch-usable code
  
The German Traffic Sign Recognition Benchmark (GTSRB) is a dataset with more than 50,000 photos of road signs from about 40 classes.  
The final aim is to recognise them !  
Description is available there : http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset


## What we're going to do :

Our main steps:
 - Try n models with n datasets
 - Save a Pandas/h5 report
 - Write to be run in batch mode

## Step 1 - Import and init
### 1.1 - Python stuffs

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import h5py
import sys,os,time,json
import random
from IPython.display import display
sys.path.append('..')
import fidle.pwk as pwk

VERSION='1.6'

sys.path.append('..')
import fidle.pwk as ooo

run_dir = './run/GTSRB5'
datasets_dir = pwk.init('GTSRB5', run_dir)

### 1.2 - Parameters
**Note:**   
With a dataset of 20% and a scale of 0.1, it takes only 2-3' of a laptop...  
With a dataset of 100% and a scale of 1, it takes 30' on a V100 GPU !  

In [ ]:
enhanced_dir = f'./data'
# enhanced_dir = f'{datasets_dir}/GTSRB/enhanced'

# ---- For tests
datasets     = ['set-24x24-L', 'set-24x24-RGB', 'set-48x48-RGB']
models       = {'v1':'get_model_v1', 'v2':'get_model_v2', 'v3':'get_model_v3'}
batch_size   = 64
epochs       = 5
scale        = 0.1
with_datagen = False
verbose      = 0

# ---- All possibilities
# datasets     = ['set-24x24-L', 'set-24x24-RGB', 'set-48x48-L', 'set-48x48-RGB', 'set-24x24-L-LHE', 'set-24x24-RGB-HE', 'set-48x48-L-LHE', 'set-48x48-RGB-HE']
# models       = {'v1':'get_model_v1', 'v2':'get_model_v2', 'v3':'get_model_v3'}
# batch_size   = 64
# epochs       = 16
# scale        = 1
# with_datagen = False
# verbose      = 0

# ---- Data augmentation
# datasets     = ['set-48x48-RGB']
# models       = {'v2':'get_model_v2'}
# batch_size   = 64
# epochs       = 20
# scale        = 1
# with_datagen = True
# verbose      = 0

Override parameters (batch mode) - Just forget this cell

In [ ]:
pwk.override('enhanced_dir', 'datasets', 'models', 'batch_size', 'epochs', 'scale', 'with_datagen', 'verbose')

## Step 2 - Start

In [ ]:
random.seed(time.time())

# ---- Where I am ?
now    = time.strftime("%A %d %B %Y - %Hh%Mm%Ss")
here   = os.getcwd()
tag_id = '{:06}'.format(random.randint(0,99999))

# ---- Who I am ?
oar_id   = os.getenv("OAR_JOB_ID",  "??")
slurm_id = os.getenv("SLURM_JOBID", "??")

print('Full Convolutions Notebook :')
print('  Version            : ', VERSION  )
print('  Now is             : ', now      )
print('  OAR id             : ', oar_id   )
print('  SLURM id           : ', slurm_id )
print('  Tag id             : ', tag_id   )
print('  Working directory  : ', here     )
print('  Output  directory  : ', run_dir  )
print('  for tensorboard    : ', f'--logdir {run_dir}')

In [ ]:
# ---- Uncomment for batch tests
#
# print("\n\n*** Test mode - Exit before making big treatments... ***\n\n")
# sys.exit()

## Step 3 - Dataset loading

In [ ]:
def read_dataset(enhanced_dir, dataset_name):
    '''Reads h5 dataset from dataset_dir
    Args:
        dataset_dir : datasets dir
        name        : dataset name, without .h5
    Returns:    x_train,y_train,x_test,y_test data'''
    # ---- Read dataset
    filename = f'{enhanced_dir}/{dataset_name}.h5'
    size     = os.path.getsize(filename)/(1024*1024)

    with  h5py.File(filename,'r') as f:
        x_train = f['x_train'][:]
        y_train = f['y_train'][:]
        x_test  = f['x_test'][:]
        y_test  = f['y_test'][:]

    # ---- Shuffle
    x_train,y_train=pwk.shuffle_np_dataset(x_train,y_train)

    # ---- done
    return x_train,y_train,x_test,y_test,size

## Step 4 - Models collection

In [ ]:

# A basic model
#
def get_model_v1(lx,ly,lz):
    
    model = keras.models.Sequential()
    
    model.add( keras.layers.Conv2D(96, (3,3), activation='relu', input_shape=(lx,ly,lz)))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Conv2D(192, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Flatten()) 
    model.add( keras.layers.Dense(1500, activation='relu'))
    model.add( keras.layers.Dropout(0.5))

    model.add( keras.layers.Dense(43, activation='softmax'))
    return model
    
# A more sophisticated model
#
def get_model_v2(lx,ly,lz):
    model = keras.models.Sequential()

    model.add( keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(lx,ly,lz), activation='relu'))
    model.add( keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add( keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Conv2D(256, (3, 3), padding='same',activation='relu'))
    model.add( keras.layers.Conv2D(256, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Flatten())
    model.add( keras.layers.Dense(512, activation='relu'))
    model.add( keras.layers.Dropout(0.5))
    model.add( keras.layers.Dense(43, activation='softmax'))
    return model

def get_model_v3(lx,ly,lz):
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (5, 5), padding='same',  activation='relu', input_shape=(lx,ly,lz)))
    model.add(tf.keras.layers.BatchNormalization(axis=-1))      
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same',  activation='relu'))
    model.add(tf.keras.layers.BatchNormalization(axis=-1))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization(axis=-1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))

    model.add(tf.keras.layers.Dense(43, activation='softmax'))
    return model

## Step 5 - Multiple datasets, multiple models ;-)

In [ ]:
def multi_run(enhanced_dir, datasets, models, datagen=None,
              scale=1, batch_size=64, epochs=16, 
              verbose=0, tag_id='last'):
    """
    Launches a dataset-model combination
    args:
        enhanced_dir   : Directory of the enhanced datasets
        datasets       : List of dataset (whitout .h5)
        models         : List of model like { "model name":get_model(), ...}
        datagen        : Data generator or None (None)
        scale          : % of dataset to use.  1 mean all. (1)
        batch_size     : Batch size (64)
        epochs         : Number of epochs (16)
        verbose        : Verbose level (0)
        tag_id         : postfix for report, logs and models dir (_last)
    return:
        report        : Report as a dict for Pandas.
    """  
    # ---- Logs and models dir
    #
    os.makedirs(f'{run_dir}/logs_{tag_id}',   mode=0o750, exist_ok=True)
    os.makedirs(f'{run_dir}/models_{tag_id}', mode=0o750, exist_ok=True)
    
    # ---- Columns of output
    #
    output={}
    output['Dataset'] = []
    output['Size']    = []
    for m in models:
        output[m+'_Accuracy'] = []
        output[m+'_Duration'] = []

    # ---- Let's go
    #
    for d_name in datasets:
        print("\nDataset : ",d_name)

        # ---- Read dataset
        x_train,y_train,x_test,y_test, d_size = read_dataset(enhanced_dir, d_name)
        output['Dataset'].append(d_name)
        output['Size'].append(d_size)
        
        # ---- Rescale
        x_train,y_train,x_test,y_test = pwk.rescale_dataset(x_train,y_train,x_test,y_test, scale=scale)
        
        # ---- Get the shape
        (n,lx,ly,lz) = x_train.shape

        # ---- For each model
        for m_name,m_function in models.items():
            print("    Run model {}  : ".format(m_name), end='')
            # ---- get model
            try:
                # ---- get function by name
                m_function=globals()[m_function]
                model=m_function(lx,ly,lz)
                # ---- Compile it
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                # ---- Callbacks tensorboard
                log_dir = f'{run_dir}/logs_{tag_id}/tb_{d_name}_{m_name}'
                tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
                # ---- Callbacks bestmodel
                save_dir = f'{run_dir}/models_{tag_id}/model_{d_name}_{m_name}.h5'
                bestmodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, monitor='accuracy', save_best_only=True)
                # ---- Train
                start_time = time.time()
                if datagen==None:
                    # ---- No data augmentation (datagen=None) --------------------------------------
                    history = model.fit(x_train, y_train,
                                        batch_size      = batch_size,
                                        epochs          = epochs,
                                        verbose         = verbose,
                                        validation_data = (x_test, y_test),
                                        callbacks       = [tensorboard_callback, bestmodel_callback])
                else:
                    # ---- Data augmentation (datagen given) ----------------------------------------
                    datagen.fit(x_train)
                    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                                        steps_per_epoch = int(len(x_train)/batch_size),
                                        epochs          = epochs,
                                        verbose         = verbose,
                                        validation_data = (x_test, y_test),
                                        callbacks       = [tensorboard_callback, bestmodel_callback])
                    
                # ---- Result
                end_time = time.time()
                duration = end_time-start_time
                accuracy = max(history.history["val_accuracy"])*100
                #
                output[m_name+'_Accuracy'].append(accuracy)
                output[m_name+'_Duration'].append(duration)
                print(f"Accuracy={accuracy: 7.2f}    Duration={duration: 7.2f}")
            except:
                raise
                output[m_name+'_Accuracy'].append('0')
                output[m_name+'_Duration'].append('999')
                print('-')
    return output

## Step 6 - Run !

In [ ]:
pwk.chrono_start()

print('\n---- Run','-'*50)


# ---- Data augmentation or not
#
if with_datagen :
    datagen = keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
                                                           featurewise_std_normalization=False,
                                                           width_shift_range=0.1,
                                                           height_shift_range=0.1,
                                                           zoom_range=0.2,
                                                           shear_range=0.1,
                                                           rotation_range=10.)
else:
    datagen=None
    
# ---- Run
#
output = multi_run(enhanced_dir,
                   datasets, 
                   models,
                   datagen       = datagen,
                   scale         = scale,
                   batch_size    = batch_size,
                   epochs        = epochs,
                   verbose       = verbose,
                   tag_id        = tag_id)

# ---- Save report
#
report={}
report['output']=output
report['description'] = f'scale={scale} batch_size={batch_size} epochs={epochs} data_aug={with_datagen}'

report_name=f'{run_dir}/report_{tag_id}.json'

with open(report_name, 'w') as file:
    json.dump(report, file, indent=4)

print('\nReport saved as ',report_name)

pwk.chrono_show()
print('-'*59)


## Step 7 - That's all folks..

In [ ]:
pwk.end()

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>